In [ ]:
%load_ext pytest_notebook.ipy_magic


In [ ]:
%%pytest --color=yes --show-capture=no --disable-warnings --nb-test-files
def add(x, y):
    return x + y

def test_add():
    assert add(1, 2) == 3
    assert add("boston", "university") == "bostonuniversity"


test_add()